# [Projet 3: Développez un moteur de recommandations de films](https://openclassrooms.com/projects/developpez-un-moteur-de-recommandations-de-films)

Link to raw data: [here](https://s3-eu-west-1.amazonaws.com/static.oc-static.com/prod/courses/files/Parcours_data_scientist/Projet+-+Moteur+de+recommandation+de+films/imdb-5000-movie-dataset.zip).

In [1]:
import os
HOME = os.path.expanduser('~/')
HOST = os.uname()[1]
if HOST == 'Arthurs-MacBook-Pro.local':
    os.chdir(HOME+'/Documents/GitHub/OCDataSciencePath/Project3/')    # @home
elif HOST == 'Sirius.local':
    os.chdir(HOME+'Perso/GitHub/OCDataSciencePath/Project3/')         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
import numpy as np
import pandas as pd

In [2]:
if HOST == 'Arthurs-MacBook-Pro.local':
    pathToData = HOME+'xxx'    # @home
elif HOST == 'Sirius.local':
    pathToData = HOME+'Downloads/movie_metadata.csv'         # @L2
else:
    raise ValueError('unknown host: {}'.format(HOST))
    
df = pd.read_csv(pathToData)

In [3]:
n,p = df.shape
print(n,p)
df.sample(5)

5043 28


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
1928,Color,Dwight H. Little,94.0,97.0,27.0,522.0,Salli Richardson-Whitfield,601.0,31526393.0,Action|Adventure|Horror|Thriller,...,144.0,English,USA,PG-13,25000000.0,2004.0,543.0,4.6,2.35,686
3716,Black and White,Christopher Nolan,274.0,113.0,22000.0,379.0,Thomas Lennon,716.0,25530884.0,Mystery|Thriller,...,2067.0,English,USA,R,9000000.0,2000.0,651.0,8.5,2.35,40000
3179,Color,David Lynch,143.0,112.0,0.0,201.0,Richard Farnsworth,874.0,6197866.0,Biography|Drama,...,414.0,English,France,G,10000000.0,1999.0,262.0,8.0,2.35,0
1459,Color,Mike Hodges,100.0,111.0,24.0,455.0,William Hootkins,591.0,NaN,Action|Adventure|Sci-Fi,...,247.0,English,UK,PG,20000000.0,1980.0,488.0,6.4,2.35,12000
1569,Black and White,Steven Soderbergh,183.0,105.0,0.0,308.0,Beau Bridges,845.0,1304837.0,Drama|Mystery|Thriller,...,175.0,English,USA,R,32000000.0,2006.0,552.0,6.1,1.66,559


In [4]:
for col in df.columns.sort_values():
    miss = 100 * (1 - df[col].count()/n)
    print('{:>30}: {}%'.format(col, miss))

        actor_1_facebook_likes: 0.13880626611144642%
                  actor_1_name: 0.13880626611144642%
        actor_2_facebook_likes: 0.25778306563553066%
                  actor_2_name: 0.25778306563553066%
        actor_3_facebook_likes: 0.4560777315090192%
                  actor_3_name: 0.4560777315090192%
                  aspect_ratio: 6.52389450723776%
                        budget: 9.756097560975608%
     cast_total_facebook_likes: 0.0%
                         color: 0.376759865159626%
                content_rating: 6.008328375966688%
                       country: 0.09914733293674427%
       director_facebook_likes: 2.0622645250842786%
                 director_name: 2.0622645250842786%
                      duration: 0.2974419988102328%
          facenumber_in_poster: 0.25778306563553066%
                        genres: 0.0%
                         gross: 17.52924846321634%
                    imdb_score: 0.0%
                      language: 0.2379535990481907%
     

We start with simple variables:

In [12]:
keepVar = ['movie_title',
           'actor_1_name','actor_2_name','actor_3_name']
wdf = df[keepVar].copy()
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name
767,The Lego Movie,Morgan Freeman,Will Ferrell,Alison Brie
3069,Poltergeist,Heather O'Rourke,Zelda Rubinstein,Craig T. Nelson
2532,Fur: An Imaginary Portrait of Diane Arbus,Robert Downey Jr.,Ty Burrell,Matt Servitto
1252,Autumn in New York,J.K. Simmons,Sam Trammell,Elaine Stritch
1009,The Life Aquatic with Steve Zissou,Bill Murray,Anjelica Huston,Matthew Gray Gubler


Needed post-processing:
- lowerise all text to avoid missing doublons
- doublons/approximative doublons in `movie_title`,`actor_N_name`
- try one-hot encode the three `actor_N_name` (ignore order)


In [13]:
# lowerise

toLowerVar = ('movie_title','actor_1_name','actor_2_name','actor_3_name')
for col in toLowerVar:
    wdf[col] = wdf[col].str.lower()
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name
3056,army of darkness,patricia tallman,bridget fonda,embeth davidtz
528,the rock,nicolas cage,michael biehn,bokeem woodbine
4819,revolution,billy burke,tracy spiridakos,david lyons
4185,escape from the planet of the apes,roddy mcdowall,m. emmet walsh,sal mineo
3137,invasion u.s.a.,billy drago,richard lynch,eddie jones


In [14]:
# remove pure duplicates

print(wdf.duplicated().sum())
# wdf.loc[wdf.duplicated()].sort_values(by='movie_title')
wdf.drop_duplicates(inplace=True)

123


In [15]:
# look specifically at the rows where only film title is duplicated

print(wdf['movie_title'].duplicated(keep=False).sum())
wdf.loc[wdf['movie_title'].duplicated(keep=False)].sort_values(by='movie_title')

6


,movie_title,actor_1_name,actor_2_name,actor_3_name
1852,brothers,natalie portman,jake gyllenhaal,bailee madison
2882,brothers,natalie portman,jake gyllenhaal,ethan suplee
3791,out of the blue,raymond burr,don gordon,jim byrnes
3840,out of the blue,william kircher,matthew sunderland,paul glover
1002,the host,j.d. evermore,chandler canterbury,rachel roberts
2988,the host,doona bae,kang-ho song,ah-sung ko


In [16]:
# no obvious error: don't want to check which one is correct... just drop them

wdf.drop_duplicates(subset='movie_title',inplace=True)
wdf.sample(5)

,movie_title,actor_1_name,actor_2_name,actor_3_name
1915,the sisterhood of the traveling pants,mike vogel,america ferrera,kyle schmid
4327,wings,steven weber,tim daly,amy yasbeck
1333,frankenweenie,martin landau,catherine o'hara,atticus shaffer
2018,star trek iv: the voyage home,leonard nimoy,nichelle nichols,walter koenig
4164,journey to saturn,iben hjejle,frank hvam,casper christensen


In [17]:
# one hot encode the names

toOneHotVar = ('actor_1_name','actor_2_name','actor_3_name')
allActor = []
for col in toOneHotVar:
    allActor += list(wdf[col])
allActor = np.unique(allActor)

In [18]:
for i,a in enumerate(allActor): # speed this up!
    value = np.full((wdf.shape[0],),False)
    for col in toOneHotVar:
        value = value & (wdf[col] == a)    
    wdf[i] = 1*value

In [19]:
wdf.drop(list(toOneHotVar),axis=1,inplace=True)
wdf.sample(5)

,movie_title,0,1,2,3,4,5,6,7,8,...,6241,6242,6243,6244,6245,6246,6247,6248,6249,6250
91,the polar express,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3874,sexy beast,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2691,knockaround guys,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2161,jaws 2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3455,ta ra rum pum,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Dump the table

In [ ]:
path,nameext= os.path.split(pathToData)
name,ext = os.path.splitext(nameext)

wdf.to_csv(os.path.join(path,name+'_CLEAN'+ext))